In [1]:
import numpy as np
import os
import utils
import BackTester
import Analysis
from typing import Dict, List

directory = '/Users/nnn/GitHub/DataStore'

path_data = os.path.join(directory, 'merge.json')
path_idx = os.path.join(directory, 'indices_data.json')

ins_analy = Analysis.AnalysisManager()

print('merge_data 로딩 시작.')
merge_data = utils._load_json(file_path=path_data)
print(f'merge_data 로딩완료 - {len(merge_data)}')
print('indices_data 로딩시작.')
indices_data = utils._load_json(file_path=path_idx)
print(f'indices_data 로딩완료 - {len(indices_data)}')

symbols = ['NOTUSDT', 'TIAUSDT', 'FETUSDT', 'XLMUSDT', 'SANDUSDT', 'DOTUSDT', 'ALGOUSDT']

obj = BackTester.IntervalKlineExtractor(symbols=symbols, kline_data=merge_data, index_data=indices_data)
obj.intervals = ['1m', '5m', '1h']
# dummy= obj.generate_real_time_kline(indices_data=indices_data, kline_data=merge_data)

merge_data 로딩 시작.
merge_data 로딩완료 - 7
indices_data 로딩시작.
indices_data 로딩완료 - 9000


In [2]:
kline_data = ins_analy.convert_kline_array(kline_data=merge_data)

In [3]:
real_data = obj.generate_real_time_kline(indices_data=indices_data, kline_data=kline_data)

In [4]:
real_5m = real_data.get('FETUSDT').get('5m')

In [5]:
target_data_5m = merge_data.get('FETUSDT').get('5m')[1234]

In [6]:
target_data_5m

[1732185000000,
 '1.20000000',
 '1.20700000',
 '1.19900000',
 '1.20500000',
 '122568.70000000',
 1732185299999,
 '147555.49060000',
 934,
 '63904.90000000',
 '76991.73800000',
 '0']

In [7]:
idxs= np.where(real_5m[:,0]==1732185000000)

In [8]:
idxs

(array([5490, 5491, 5492, 5493, 5494]),)

In [12]:
real_5m[5494][5] - float(target_data_5m[5])

0.0

In [10]:
real_5m[idxs[-1]]

array([[1.73218500e+12, 1.20000000e+00, 1.20600000e+00, 1.19900000e+00,
        1.20600000e+00, 4.47572000e+04, 1.73218530e+12, 5.37720900e+04,
        1.84000000e+02, 1.99513000e+04, 2.40115238e+04, 0.00000000e+00],
       [1.73218500e+12, 1.20000000e+00, 1.20700000e+00, 1.19900000e+00,
        1.20600000e+00, 8.58491000e+04, 1.73218530e+12, 1.03317371e+05,
        5.23000000e+02, 3.90225000e+04, 4.70114909e+04, 0.00000000e+00],
       [1.73218500e+12, 1.20000000e+00, 1.20700000e+00, 1.19900000e+00,
        1.20400000e+00, 1.01809900e+05, 1.73218530e+12, 1.22553871e+05,
        6.76000000e+02, 4.99295000e+04, 6.01585786e+04, 0.00000000e+00],
       [1.73218500e+12, 1.20000000e+00, 1.20700000e+00, 1.19900000e+00,
        1.20500000e+00, 1.16140800e+05, 1.73218530e+12, 1.39808637e+05,
        7.82000000e+02, 5.98164000e+04, 7.20633734e+04, 0.00000000e+00],
       [1.73218500e+12, 1.20000000e+00, 1.20700000e+00, 1.19900000e+00,
        1.20500000e+00, 1.22568700e+05, 1.73218530e+12, 1.47

In [13]:
real_data

{'DOTUSDT': {'1m': array([[1.73185560e+12, 5.57700000e+00, 5.57700000e+00, ...,
          1.85563000e+03, 1.03440541e+04, 0.00000000e+00],
         [1.73185566e+12, 5.57100000e+00, 5.57700000e+00, ...,
          2.96741000e+03, 1.65398459e+04, 0.00000000e+00],
         [1.73185572e+12, 5.57500000e+00, 5.57600000e+00, ...,
          1.37754700e+04, 7.66598359e+04, 0.00000000e+00],
         ...,
         [1.73239542e+12, 8.48800000e+00, 8.48800000e+00, ...,
          3.65465000e+03, 3.09602608e+04, 0.00000000e+00],
         [1.73239548e+12, 8.47500000e+00, 8.50400000e+00, ...,
          6.30367000e+03, 5.35213394e+04, 0.00000000e+00],
         [1.73239554e+12, 8.49800000e+00, 8.50000000e+00, ...,
          4.12504000e+03, 3.50131363e+04, 0.00000000e+00]]),
  '5m': array([[1.73185560e+12, 5.57700000e+00, 5.57700000e+00, ...,
          1.85563000e+03, 1.03440541e+04, 0.00000000e+00],
         [1.73185560e+12, 5.57700000e+00, 5.57700000e+00, ...,
          4.82304000e+03, 2.68839000e+04, 0.